## 张量和操作

### 导入TF

在开始前，先导入tf2，在tf2中，动态图默认开启，这使得tf2的前端更具交互性，我们将在后面详细讨论。

In [1]:
import tensorflow as tf

## 张量
张量本质上就是一个多维矩阵,类似于Numpy的`ndarray`类型,`tf.Tensor`类型拥有数据类型和维度.另外`tf.Tensor`类型数据可以在GPU中进行加速,TF提供了很多计算和创造tensor的库,这些操作可以自动的转换到原生的Python类型,例如:

In [2]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


每一个`tf.Tensor`都有维度和数据类型

In [3]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


numpy数组和tf.Tensor最明显的区别是:

1. 张量能否使用GPU等加速

2. 张量是不变的 

## Numpy兼容

在`tf.Tensor`和Numpy`ndarray`中转换是容易的:
    - TF会自动将ndarray转换为Tensor
    - Numpy操作会自动将Tensor转换为ndarray

张量使用它们的`.numpy()`方法显式转换为NumPy ndarray.由于数组和`tf.tensor`共享底层内存表示。但是，共享底层表示并不总是可能的，因为`tf.tensor`可能托管在GPU内存中，而NumPy数组始终由主机内存支持，并且转换涉及从GPU到主机内存的副本。

In [4]:
import numpy as np
ndarray = np.ones([3, 3])
print('tf自动转换ndarray到tensor')
tensor = tf.multiply(ndarray, 42)
print(tensor)

print('numpy自动转换tensor到ndarray')
print(np.add(tensor, 1))

print('使用.numpy函数将tensor转换为numpy array')
print(tensor.numpy())

tf自动转换ndarray到tensor
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
numpy自动转换tensor到ndarray
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
使用.numpy函数将tensor转换为numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU加速
许多tf操作可以通过GPU来加速计算,在没有任何注释的情况下，如果必要的话,TensorFlow会自动决定是否使用GPU还是CPU来执行在CPU和GPU内存之间复制张量的操作。操作产生的张量通常由执行操作的设备的内存支持，例如：

In [6]:
# 均匀分布
x = tf.random.uniform([3,3])
print('GPU 可用')
print(tf.config.experimental.list_physical_devices('GPU'))

print('Tensor on GPU #0: ')
print(x.device.endswith('GPU:0'))

GPU 可用
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Tensor on GPU #0: 
True


## 设备名称

`Tensor.device`属性提供了承载张量内容的设备的完整的名称和地址.此名称对很多细节进行了编码,像网络地址标识符以及在该主机上执行的设备,这是tf分布式程序所必需的.这个字符串使用`GPU:<N>`结尾,如果tensor放置在了`N`-th GPU上,例如:

## 显式设备指定

在tf中,如果没有指定设备,tf会自动决定使用哪个设备进行操作,如果要指定设备进行操作的话,可以使用`tf.device`进行显式设备指定

In [7]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
    
    result = time.time()-start
    print('10 loops: {:0.2f}ms'.format(1000*result))

print('On CPU:')

with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith('CPU:0')
    time_matmul(x)

if tf.config.experimental.list_physical_devices('GPU'):
    print('On GPU:')
    with tf.device('GPU:0'):
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith('GPU:0')
        time_matmul(x)

On CPU:
10 loops: 68.30ms
On GPU:
10 loops: 211.94ms
